In [6]:
import pandas as pd, numpy as np
from sklearn.datasets import load_boston

In [1]:
boston = load_boston()

In [21]:
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['label'] = [1 if t > np.mean(boston.target) else 0 for t in boston.target]

In [22]:
df.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,label
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,1
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,0
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,1
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,1
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,1


In [57]:
def calculate_fishers(data):
    X = data.drop()'label',axis=1)
    y = data['label']
    
    # calculate mean of each feature
    means = np.array(np.mean(X))
    # calculate mean of all data
    mean_all = np.array(np.mean(means))
    
    # between class covariance matrix
    M = means - mean_all
    S_B = np.sum(np.outer(M, np.transpose(M)))
    
    # within class covariance matrix
    temp = X - np.array(np.mean(X))
    S_W = np.sum(np.sum(np.dot(temp, np.transpose(temp))))

SyntaxError: invalid syntax (<ipython-input-57-e7dc8d138eb7>, line 2)

In [76]:
means = {}
for c in X.columns:
    means[c] = np.array(X[c].mean(axis=0))
overall_mean = np.array(X.mean(axis=0))
S_B = np.zeros((X.shape[1], X.shape[1]))
for c in means.keys():
    S_B += np.multiply(len(X[c]),
                      np.outer((means[c] - overall_mean),
                              np.transpose(means[c] - overall_mean)))

In [80]:
np.array(X.mean(axis=0))

array([3.59376071e+00, 1.13636364e+01, 1.11367787e+01, 6.91699605e-02,
       5.54695059e-01, 6.28463439e+00, 6.85749012e+01, 3.79504269e+00,
       9.54940711e+00, 4.08237154e+02, 1.84555336e+01, 3.56674032e+02,
       1.26530632e+01])

In [79]:
np.array(np.mean(X))

array([3.59376071e+00, 1.13636364e+01, 1.11367787e+01, 6.91699605e-02,
       5.54695059e-01, 6.28463439e+00, 6.85749012e+01, 3.79504269e+00,
       9.54940711e+00, 4.08237154e+02, 1.84555336e+01, 3.56674032e+02,
       1.26530632e+01])

In [25]:
X = df.drop('label',axis=1)
y = df['label']

In [50]:
np.mean(means)

70.07244682578286

In [81]:
# calculate mean of each feature
means = np.array(np.mean(X))
# calculate mean of all data
mean_all = np.array(np.mean(np.mean(X)))
    
# between class covariance matrix
M = means - mean_all
S_B = np.sum(np.outer(M, np.transpose(M)))
S_B

-9.094947017729282e-12

In [68]:
temp = X - np.array(np.mean(X))
np.sum(np.dot(temp, np.transpose(temp)))

5.075708031654358e-08

In [40]:
np.multiply(A, np.transpose(A))

array([4.41941571e+03, 3.44672443e+03, 3.47341298e+03, 4.90045877e+03,
       4.83271781e+03, 4.06888502e+03, 2.24264294e+00, 4.39269430e+03,
       3.66303834e+03, 1.14355369e+05, 2.66430573e+03, 8.21404684e+04,
       3.29698561e+03])